In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


# Bug itself

Following plot illustrates bug perfectly. It's plot of incomplete beta for veryy small range 

In [ ]:
pts = [ ( i
        , fromIntegral (ulpDistance y0 $ incompleteBeta 0.5 0.5 (addUlps (fromIntegral i) x0)) :: Int
        )
      | i <- [0 .. 200]]
  where
    x0 = 1e-10
    y0 = incompleteBeta 0.5 0.5 x0

toRenderable
  $ layout_plots .~ 
      [ toPlot $ plot_lines_values .~ [pts] $ def]
  $ def 


Let copy sourse code here yet again

In [ ]:
-- | Regularized incomplete beta function. Uses algorithm AS63 by
-- Majumder and Bhattachrjee and quadrature approximation for large
-- /p/ and /q/.
incompleteBeta' :: Double -- ^ /p/ > 0
               -> Double -- ^ /q/ > 0
               -> Double -- ^ /x/, must lie in [0,1] range
               -> Double
incompleteBeta' p q = incompleteBeta_ (logBeta p q) p q

-- | Regularized incomplete beta function. Same as 'incompleteBeta'
-- but also takes logarithm of beta function as parameter.
incompleteBeta_ :: Double -- ^ logarithm of beta function for given /p/ and /q/
                -> Double -- ^ /p/ > 0
                -> Double -- ^ /q/ > 0
                -> Double -- ^ /x/, must lie in [0,1] range
                -> Double
incompleteBeta_ beta p q x
  | p <= 0 || q <= 0            =
      error $ printf "incompleteBeta_: p <= 0 || q <= 0. p=%g q=%g x=%g" p q x
  | x <  0 || x >  1 || isNaN x =
      error $ printf "incompletBeta_: x out of [0,1] range. p=%g q=%g x=%g" p q x
  | x == 0 || x == 1            = x
  | p >= (p+q) * x   = incompleteBetaWorker beta p q x
  | otherwise        = 1 - incompleteBetaWorker beta q p (1 - x)


-- Worker for incomplete beta function. It is separate function to
-- avoid confusion with parameter during parameter swapping
incompleteBetaWorker :: Double -> Double -> Double -> Double -> Double
incompleteBetaWorker beta p q x
  -- For very large p and q this method becomes very slow so another
  -- method is used.
  | p > 3000 && q > 3000 = error "removed"
  | otherwise            = loop (p+q) (truncate $ q + cx * (p+q)) 1 1 1
  where
    -- Constants
    eps = 1e-18
    cx  = 1 - x
    -- Loop
    loop !psq (ns :: Int) ai term betain
      | done      = betain' * exp( p * log x + (q - 1) * log cx - beta) / p
      | otherwise = loop psq' (ns - 1) (ai + 1) term' betain'
      where
        -- New values
        term'   = term * fact / (p + ai)
        betain' = betain + term'
        fact | ns >  0   = (q - ai) * x/cx
             | ns == 0   = (q - ai) * x
             | otherwise = psq * x
        -- Iterations are complete
        done = db <= eps && db <= eps*betain' where db = abs term'
        psq' = if ns < 0 then psq + 1 else psq


In [ ]:
pts = [ ( i
        , fromIntegral (ulpDistance y0 $ incompleteBeta' 0.5 0.5 (addUlps (fromIntegral i) x0)) :: Int
        )
      | i <- [0 .. 200]]
  where
    x0 = 1e-10
    y0 = incompleteBeta 0.5 0.5 x0

toRenderable
  $ layout_plots .~ 
      [ toPlot $ plot_lines_values .~ [pts] $ def]
  $ def 
